<a href="https://colab.research.google.com/github/akshaydeepakm44/Emotion-detection-Audio-/blob/main/Copy_of_Emotion_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The data used in the proejct is
speech-emotion-recognition-ravdess-data

In [ ]:
pip install librosa numpy scikit-learn


In [ ]:
!pip uninstall -y resampy librosa
!pip install resampy librosa


Found existing installation: resampy 0.4.3
Uninstalling resampy-0.4.3:
  Successfully uninstalled resampy-0.4.3
Found existing installation: librosa 0.10.2.post1
Uninstalling librosa-0.10.2.post1:
  Successfully uninstalled librosa-0.10.2.post1
  Using cached resampy-0.4.3-py3-none-any.whl.metadata (3.0 kB)
  Using cached librosa-0.10.2.post1-py3-none-any.whl.metadata (8.6 kB)
Using cached resampy-0.4.3-py3-none-any.whl (3.1 MB)
Using cached librosa-0.10.2.post1-py3-none-any.whl (260 kB)


In [ ]:
import librosa
import resampy

audio_path = '/content/speech-emotion-recognition-ravdess-data.wav'

# Test librosa's load function
audio, sample_rate = librosa.load(audio_path, sr=None)
print("Audio Loaded Successfully!")
print("Audio Shape:", audio.shape)
print("Sample Rate:", sample_rate)


Audio Loaded Successfully!
Audio Shape: (52853,)
Sample Rate: 16000


In [ ]:
def extract_features(file_path):
    try:
        print(f"Loading audio file: {file_path}")
        audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
        print(f"Audio loaded: {len(audio)} samples at {sample_rate} Hz")

        mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40).T, axis=0)
        print("MFCCs computed.")

        chroma = np.mean(librosa.feature.chroma_stft(y=audio, sr=sample_rate).T, axis=0)
        print("Chroma features computed.")

        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=audio, sr=sample_rate).T, axis=0)
        print("Spectral contrast computed.")

        return np.hstack([mfccs, chroma, spectral_contrast])
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None


In [ ]:
def extract_features(file_path):
    try:
        audio, sample_rate = librosa.load(file_path, sr=None)
        mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40).T, axis=0)
        chroma = np.mean(librosa.feature.chroma_stft(y=audio, sr=sample_rate).T, axis=0)
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=audio, sr=sample_rate).T, axis=0)

        features = np.hstack([mfccs, chroma, spectral_contrast])
        print(f"Feature dimension for {file_path}: {features.shape}")
        return features
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None


In [ ]:
from scipy.io.wavfile import read

def load_audio_with_scipy(file_path):
    try:
        sample_rate, audio = read(file_path)
        print(f"Audio loaded successfully using scipy: {audio.shape}, {sample_rate} Hz")
        return audio, sample_rate
    except Exception as e:
        print(f"Error loading audio with scipy: {e}")
        return None, None


In [ ]:
import librosa
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Mock dataset
emotions = ['neutral', 'happy', 'sad', 'angry']
mock_features = np.random.rand(200, 59)  # 200 samples, 59 features
mock_labels = np.array([i // 50 for i in range(200)])  # 50 samples per emotion

# Train/Test split
x_train, x_test, y_train, y_test = train_test_split(mock_features, mock_labels, test_size=0.2, random_state=42)

# Train Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(x_train, y_train)

# Feature extraction function
def extract_features(file_path):
    try:
        audio, sample_rate = librosa.load(file_path, sr=None)
        mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40).T, axis=0)
        chroma = np.mean(librosa.feature.chroma_stft(y=audio, sr=sample_rate).T, axis=0)
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=audio, sr=sample_rate).T, axis=0)
        features = np.hstack([mfccs, chroma, spectral_contrast])
        print(f"Feature dimension for {file_path}: {features.shape}")
        return features
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

# Function to pad or truncate features
def pad_or_truncate(features, target_length):
    if len(features) < target_length:
        features = np.pad(features, (0, target_length - len(features)), mode='constant')
    elif len(features) > target_length:
        features = features[:target_length]
    return features

# Path to audio file
audio_file_path = '/content/speech-emotion-recognition-ravdess-data.wav'

# Extract features
features = extract_features(audio_file_path)
if features is None:
    raise ValueError("Feature extraction failed. Please check the audio file.")

# Adjust feature length
target_length = 59  # Match the mock dataset
features = pad_or_truncate(features, target_length)

# Predict emotion
features = features.reshape(1, -1)  # Reshape for single sample
predicted_emotion = model.predict(features)
predicted_emotion_label = emotions[predicted_emotion[0]]
print("Predicted Emotion:", predicted_emotion_label)


Feature dimension for /content/03-01-01-01-01-01-01.wav: (59,)
Predicted Emotion: happy
